Link to HW is [here](https://github.com/hoangpham95/4400/blob/master/HW3/HW3%20official.ipynb)

# Question 1

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [2]:
# Load data
matrix = []

with open("spambase.data", "r") as raw_data:
    for raw_line in raw_data:
        line = [float(x) for x in raw_line.split(",")]
        matrix.append(line)

data = pd.DataFrame(matrix)
row, col = data.shape
X, y = data.iloc[:,:col - 1], data[col - 1]
# y = y.astype(bool)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

## Training Random Forest Classifier on existing package

In [4]:
from sklearn.ensemble import RandomForestClassifier

/home/hoang/Documents/school/4400/env/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [6]:
def classfierData(clf, X, y):
    test_data = clf.predict(X)
    print("Confusion matrix\n", confusion_matrix(test_data, y), "\n")
    tn, fp, fn, tp = confusion_matrix(test_data, y).ravel()
    print("True negative:", tn, ", false positive:", fp, ", false negative:", fn, ",true positive:", tp, "\n")
    print("Accuracy score", accuracy_score(test_data, y), "\n")
    print("Precision", precision_score(test_data, y), "\n")
    print("Recall", recall_score(test_data, y), "\n")
    print("F1 score", f1_score(test_data, y), "\n")

In [7]:
classfierData(clf, X_train, y_train)

Confusion matrix
 [[2091    2]
 [   0 1357]] 

True negative: 2091 , false positive: 2 , false negative: 0 ,true positive: 1357 

Accuracy score 0.9994202898550725 

Precision 0.9985283296541575 

Recall 1.0 

F1 score 0.9992636229749632 



In [8]:
classfierData(clf, X_test, y_test)

Confusion matrix
 [[673  37]
 [ 24 417]] 

True negative: 673 , false positive: 37 , false negative: 24 ,true positive: 417 

Accuracy score 0.947002606429192 

Precision 0.9185022026431718 

Recall 0.9455782312925171 

F1 score 0.9318435754189944 



## Implementing own Random Forest Classifier

Predicate class acts as the splitting question:

In [9]:
class Predicate:
    def __init__(self, column, value):
        self.column = column
        self.value = value
    
    def match(self, example, pp=False):
        if pp:
            print("Match on column", self.column)
        val = example[self.column]
        if Util.is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    
    def __str__(self):
        return 'Question: column {}, value {}'.format(self.column, self.value)

Some utility functions that being reused a lot in the implementation of random forest. In this implementation, I used gini impurity instead of entropy since `log` function would arguably take longer to calculate

In [10]:
import time

class Util:
    @staticmethod
    def label_count(labels):
        count = {}
        for r in labels:
            if r not in count:
                count[r] = 0
            count[r] += 1
        return count
    
    @staticmethod
    def is_numeric(val):
        return isinstance(val, int) or isinstance(val, float)
    
    @staticmethod
    def partition(X, y, pred):
        true_X, false_X, true_y, false_y = [], [], [], []
        
        for x_inst, y_inst in zip(X, y):
            if pred.match(x_inst):
                true_X.append(x_inst)
                true_y.append(y_inst)
            else:
                false_X.append(x_inst)
                false_y.append(y_inst)
        return true_X, true_y, false_X, false_y
    
    @staticmethod
    def gini_impur(labels):
        """
        Gini impurity
        """
        counts = Util.label_count(labels)
        total = 0
        for lbl in counts:
            prob_of_lbl = float(counts[lbl]) / len(labels)
            total += (prob_of_lbl * prob_of_lbl)
        return 1 - total
    
    @staticmethod
    def info_gain(leftLbl, rightLbl, curr_uncertainty):
        """
        Calculating information gain
        """
        p = float (len(leftLbl)) / (len(leftLbl) + len(rightLbl))
        return curr_uncertainty - p * Util.gini_impur(leftLbl) - (1 - p) * Util.gini_impur(rightLbl)

Tree data structure

In [11]:
class Leaf:
    def __init__(self, y, depth=0):
        self.depth = depth
        pred = Util.label_count(y)
        for l in pred:
            pred[l] = pred[l] / len(y)
        self.predictions = pred
        
    def isLeaf(self):
        return True

class DecTreeNode:
    def __init__(self, pred, true_branch, false_branch, depth=0):
        self.depth = depth
        self.pred = pred
        self.true_branch = true_branch
        self.false_branch = false_branch
        
    def isLeaf(self):
        return False

Implementation of Decision Tree

In [12]:
class DecTreeClassifier:
    def __init__(self, max_depth=10):
        self.max_depth = max_depth
    
    def findBestSplit(self, X, y, pp=False):
        start = time.time()
        best_gain = 0  
        best_pred = None
        current_uncert = Util.gini_impur(y)
        
        for col in self.selected_features:
            vals = set([row[col] for row in X]) # different values in column
            
            for v in vals:
                pred = Predicate(col, v)
                true_X, true_y, false_X, false_y = Util.partition(X, y, pred)
                
                if len(true_X) == 0 or len(false_X) == 0:
                    continue
                
                gain = Util.info_gain(true_y, false_y, current_uncert)
                
                if gain >= best_gain:
                    best_gain, best_pred = gain, pred
        if pp:
            print("Find best split size", np.array(X).shape, "took", time.time() - start)
        return best_gain, best_pred
    
    def build_tree(self, X, y, depth=0):
        gain, pred = self.findBestSplit(X, y)
        if gain == 0 or depth >= self.max_depth:
            return Leaf(y, depth)
        true_X, true_y, false_X, false_y = Util.partition(X, y, pred)
        
        true_branch = self.build_tree(true_X, true_y, depth + 1)
        false_branch = self.build_tree(false_X, false_y, depth + 1)
        
        return DecTreeNode(pred, true_branch, false_branch)
    
    def predict_by_tree(self, tree, X_inst):
        if tree.isLeaf():
            return tree.predictions
        elif tree.pred.match(X_inst, pp=False):
            return self.predict_by_tree(tree.true_branch, X_inst)
        else:
            return self.predict_by_tree(tree.false_branch, X_inst)
    
    def fit(self, X, y, selected_features=None):
        if selected_features is None:
            self.selected_features = X.columns
        else:
            self.selected_features = selected_features
            
        self.tree = self.build_tree(X.values, y.values) 
    
    def predict_inst(self, X_inst):
        pred = self.predict_by_tree(self.tree, np.array(X_inst))
        max_arg, max_prob = None, 0
        for l in pred:
            if pred[l] > max_prob:
                max_arg = l
            max_prob = max(max_prob, pred[l])
        return max_arg
    
    def predict(self, X):
        y = []
        for idx, r in X.iterrows():
            y.append(self.predict_inst(r))
        return pd.DataFrame(y)

In [13]:
# Training decision tree
decTreeClf = DecTreeClassifier(max_depth=10)
start = time.time()
decTreeClf.fit(X_train, y_train)
print("Decision tree classifier take", time.time() - start, "seconds to build tree")

Decision tree classifier take 258.04462361335754 seconds to build tree


So to build a full tree with depth 10, it took me 247 seconds (~4 minutes). Having max depth helps us limit the training time and also avoid overfitting on the training data.

In [14]:
classfierData(decTreeClf, X_test, y_test)

Confusion matrix
 [[657  54]
 [ 40 400]] 

True negative: 657 , false positive: 54 , false negative: 40 ,true positive: 400 

Accuracy score 0.9183318853171155 

Precision 0.8810572687224669 

Recall 0.9090909090909091 

F1 score 0.894854586129754 



In [15]:
from sklearn.tree import DecisionTreeClassifier
dcf = DecisionTreeClassifier(max_depth=10)
dcf.fit(X_train, y_train)
classfierData(dcf, X_test, y_test)

Confusion matrix
 [[654  54]
 [ 43 400]] 

True negative: 654 , false positive: 54 , false negative: 43 ,true positive: 400 

Accuracy score 0.9157254561251086 

Precision 0.8810572687224669 

Recall 0.9029345372460497 

F1 score 0.8918617614269787 



Hence, my implementation of Decision Tree Classifier gains approximately the same result as sklearn's Decision Tree Classifier (less than 1% different in accuracy, precision, etc.). Now we can implement random forest classifier after decision tree

In [16]:
from sklearn.utils import resample
import random

class RandomForestClassifier:
    def __init__(self, num_est=10, max_depth=5, num_attrs=5):
        self.num_est = num_est
        self.max_depth = max_depth
        self.forest = []
        self.num_attrs = num_attrs
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        
        # Bagging
        for i in range(self.num_est):
            shape = len(X.columns)
            
            selected_features = random.sample(range(0, shape), self.num_attrs)
            X_samp, y_samp = resample(X, y, replace=True)
            clf = DecTreeClassifier(max_depth=self.max_depth)
            print("Selected features", selected_features)
            start = time.time()
            print("=== Training estimator #", i + 1, "with", len(selected_features),"features")
            clf.fit(X_samp, y_samp, selected_features=selected_features)
            print("===> Done training estimator #", i + 1, "in", time.time() - start, "seconds")
            self.forest.append(clf)
            print("--------------------------------------------------------------\n")
            
    def predict(self, X):
        result = []
        for idx, x in X.iterrows():
            labels = {}
            for clf in self.forest:
                l = clf.predict_inst(x)
                if l not in labels:
                    labels[l] = 0
                labels[l] += 1
            for l in labels:
                if labels[l] >= 0.5:
                    result.append(l)
                    break
        return pd.DataFrame(result)
            

## Testing random forest on $\sqrt{d}$ attributes

In [17]:
rfc = RandomForestClassifier(num_est=10, max_depth=10, num_attrs=7)

In [18]:
rfc.fit(X_train, y_train)

Selected features [51, 26, 36, 54, 12, 4, 43]
=== Training estimator # 1 with 7 features
===> Done training estimator # 1 in 27.395561933517456 seconds
--------------------------------------------------------------

Selected features [2, 32, 1, 20, 19, 27, 47]
=== Training estimator # 2 with 7 features
===> Done training estimator # 2 in 11.714879274368286 seconds
--------------------------------------------------------------

Selected features [42, 54, 55, 46, 51, 38, 56]
=== Training estimator # 3 with 7 features
===> Done training estimator # 3 in 35.14453482627869 seconds
--------------------------------------------------------------

Selected features [35, 28, 34, 25, 50, 42, 17]
=== Training estimator # 4 with 7 features
===> Done training estimator # 4 in 10.402256488800049 seconds
--------------------------------------------------------------

Selected features [5, 13, 23, 50, 20, 46, 9]
=== Training estimator # 5 with 7 features
===> Done training estimator # 5 in 11.258687257

In [19]:
classfierData(rfc, X_test, y_test)
classfierData(rfc, X_train, y_train)

Confusion matrix
 [[629 100]
 [ 68 354]] 

True negative: 629 , false positive: 100 , false negative: 68 ,true positive: 354 

Accuracy score 0.8540399652476107 

Precision 0.7797356828193832 

Recall 0.8388625592417062 

F1 score 0.8082191780821917 

Confusion matrix
 [[1969  191]
 [ 122 1168]] 

True negative: 1969 , false positive: 191 , false negative: 122 ,true positive: 1168 

Accuracy score 0.9092753623188405 

Precision 0.8594554819720382 

Recall 0.9054263565891473 

F1 score 0.8818422046055114 



## Random forest on all attributes with 10 estimators

In [20]:
rfc = RandomForestClassifier(num_est=10, max_depth=10, num_attrs=57)
rfc.fit(X_train, y_train)
classfierData(rfc, X_test, y_test)

Selected features [6, 48, 7, 13, 30, 2, 39, 12, 16, 38, 20, 26, 11, 32, 21, 23, 0, 36, 14, 34, 33, 25, 56, 31, 29, 49, 53, 19, 27, 8, 42, 22, 54, 5, 3, 52, 44, 43, 50, 18, 17, 35, 15, 46, 37, 45, 24, 9, 51, 55, 40, 10, 47, 1, 28, 4, 41]
=== Training estimator # 1 with 57 features
===> Done training estimator # 1 in 184.23791527748108 seconds
--------------------------------------------------------------

Selected features [25, 10, 1, 33, 17, 21, 14, 12, 20, 4, 0, 51, 40, 49, 31, 27, 48, 39, 47, 56, 55, 46, 38, 13, 7, 26, 54, 24, 6, 45, 37, 29, 3, 32, 44, 53, 52, 43, 16, 22, 8, 2, 30, 15, 50, 23, 35, 18, 34, 36, 19, 9, 41, 28, 42, 11, 5]
=== Training estimator # 2 with 57 features
===> Done training estimator # 2 in 183.29053115844727 seconds
--------------------------------------------------------------

Selected features [31, 12, 51, 55, 42, 46, 30, 26, 47, 27, 2, 1, 20, 7, 52, 43, 8, 41, 35, 22, 4, 21, 0, 37, 32, 38, 3, 29, 54, 13, 33, 18, 53, 50, 17, 34, 24, 45, 11, 48, 9, 28, 5, 49

In [21]:
## With d/2 features
rfc = RandomForestClassifier(num_est=10, max_depth=10, num_attrs=int(57/2))
rfc.fit(X_train, y_train)
classfierData(rfc, X_test, y_test)

Selected features [20, 1, 14, 41, 51, 44, 17, 36, 9, 55, 6, 34, 26, 48, 13, 28, 50, 3, 25, 37, 47, 32, 43, 56, 31, 4, 21, 49]
=== Training estimator # 1 with 28 features
===> Done training estimator # 1 in 73.9450249671936 seconds
--------------------------------------------------------------

Selected features [25, 19, 12, 55, 13, 4, 37, 8, 33, 45, 31, 56, 9, 53, 36, 29, 22, 54, 40, 41, 20, 5, 17, 30, 11, 15, 16, 24]
=== Training estimator # 2 with 28 features
===> Done training estimator # 2 in 83.8211739063263 seconds
--------------------------------------------------------------

Selected features [13, 29, 0, 3, 28, 52, 19, 35, 51, 23, 9, 10, 50, 21, 48, 5, 4, 39, 44, 32, 26, 30, 55, 25, 22, 38, 56, 49]
=== Training estimator # 3 with 28 features
===> Done training estimator # 3 in 85.79024934768677 seconds
--------------------------------------------------------------

Selected features [47, 36, 15, 33, 9, 6, 5, 56, 18, 4, 14, 53, 43, 32, 52, 28, 21, 23, 3, 24, 44, 39, 51, 13, 0, 

In [22]:
rfc = RandomForestClassifier(num_est=10, max_depth=10, num_attrs=int(np.sqrt(57)))
rfc.fit(X_train, y_train)
classfierData(rfc, X_test, y_test)

Selected features [31, 42, 19, 54, 45, 35, 5]
=== Training estimator # 1 with 7 features
===> Done training estimator # 1 in 21.07187843322754 seconds
--------------------------------------------------------------

Selected features [7, 44, 22, 27, 21, 47, 10]
=== Training estimator # 2 with 7 features
===> Done training estimator # 2 in 11.313880205154419 seconds
--------------------------------------------------------------

Selected features [35, 16, 48, 43, 45, 3, 33]
=== Training estimator # 3 with 7 features
===> Done training estimator # 3 in 10.242543697357178 seconds
--------------------------------------------------------------

Selected features [26, 55, 39, 51, 11, 25, 12]
=== Training estimator # 4 with 7 features
===> Done training estimator # 4 in 18.373183488845825 seconds
--------------------------------------------------------------

Selected features [47, 15, 39, 4, 17, 9, 40]
=== Training estimator # 5 with 7 features
===> Done training estimator # 5 in 11.567167758

## Random forest with $\sqrt{d}$ attributes and varying number of estimator

My implementation of random forest returns approximately 80% of accuracy across 3 models (first with 10 estimators, next with 50 estimators and last with 100 estimators). sklearn's package returns a more performant one with nearly 98-99% accuracy.

In [23]:
rfc = RandomForestClassifier(num_est=10, max_depth=10, num_attrs=int(np.sqrt(57)))
rfc.fit(X_train, y_train)
classfierData(rfc, X_test, y_test)

print("*****************************************************************************************\n")

rfc = RandomForestClassifier(num_est=50, max_depth=10, num_attrs=int(np.sqrt(57)))
rfc.fit(X_train, y_train)
classfierData(rfc, X_test, y_test)

print("*****************************************************************************************\n")

rfc = RandomForestClassifier(num_est=100, max_depth=10, num_attrs=int(np.sqrt(57)))
rfc.fit(X_train, y_train)
classfierData(rfc, X_test, y_test)

Selected features [38, 37, 35, 26, 25, 40, 28]
=== Training estimator # 1 with 7 features
===> Done training estimator # 1 in 8.193460464477539 seconds
--------------------------------------------------------------

Selected features [36, 10, 26, 56, 52, 46, 1]
=== Training estimator # 2 with 7 features
===> Done training estimator # 2 in 15.2423574924469 seconds
--------------------------------------------------------------

Selected features [9, 14, 56, 37, 20, 8, 41]
=== Training estimator # 3 with 7 features
===> Done training estimator # 3 in 15.116879224777222 seconds
--------------------------------------------------------------

Selected features [23, 27, 41, 1, 0, 48, 37]
=== Training estimator # 4 with 7 features
===> Done training estimator # 4 in 10.245727300643921 seconds
--------------------------------------------------------------

Selected features [52, 6, 43, 46, 13, 35, 42]
=== Training estimator # 5 with 7 features
===> Done training estimator # 5 in 8.6361777782440

===> Done training estimator # 27 in 13.122714042663574 seconds
--------------------------------------------------------------

Selected features [4, 22, 55, 14, 35, 23, 50]
=== Training estimator # 28 with 7 features
===> Done training estimator # 28 in 11.358417510986328 seconds
--------------------------------------------------------------

Selected features [29, 4, 26, 49, 27, 13, 38]
=== Training estimator # 29 with 7 features
===> Done training estimator # 29 in 16.366130590438843 seconds
--------------------------------------------------------------

Selected features [4, 32, 27, 50, 11, 56, 13]
=== Training estimator # 30 with 7 features
===> Done training estimator # 30 in 18.05549931526184 seconds
--------------------------------------------------------------

Selected features [0, 21, 52, 27, 2, 41, 1]
=== Training estimator # 31 with 7 features
===> Done training estimator # 31 in 13.26807951927185 seconds
--------------------------------------------------------------

Sele

===> Done training estimator # 14 in 10.00845718383789 seconds
--------------------------------------------------------------

Selected features [48, 7, 50, 54, 0, 8, 4]
=== Training estimator # 15 with 7 features
===> Done training estimator # 15 in 25.42400312423706 seconds
--------------------------------------------------------------

Selected features [12, 6, 25, 17, 26, 29, 23]
=== Training estimator # 16 with 7 features
===> Done training estimator # 16 in 12.363424301147461 seconds
--------------------------------------------------------------

Selected features [11, 48, 47, 30, 23, 5, 4]
=== Training estimator # 17 with 7 features
===> Done training estimator # 17 in 14.610034942626953 seconds
--------------------------------------------------------------

Selected features [42, 31, 21, 17, 55, 15, 39]
=== Training estimator # 18 with 7 features
===> Done training estimator # 18 in 9.711392879486084 seconds
--------------------------------------------------------------

Select

===> Done training estimator # 52 in 7.626392602920532 seconds
--------------------------------------------------------------

Selected features [9, 7, 34, 4, 52, 20, 25]
=== Training estimator # 53 with 7 features
===> Done training estimator # 53 in 16.185197114944458 seconds
--------------------------------------------------------------

Selected features [27, 12, 46, 36, 55, 8, 10]
=== Training estimator # 54 with 7 features
===> Done training estimator # 54 in 8.752629518508911 seconds
--------------------------------------------------------------

Selected features [40, 35, 47, 11, 46, 50, 44]
=== Training estimator # 55 with 7 features
===> Done training estimator # 55 in 10.007098197937012 seconds
--------------------------------------------------------------

Selected features [47, 40, 20, 51, 13, 4, 45]
=== Training estimator # 56 with 7 features
===> Done training estimator # 56 in 18.036325693130493 seconds
--------------------------------------------------------------

Sel

===> Done training estimator # 90 in 10.079874277114868 seconds
--------------------------------------------------------------

Selected features [53, 47, 48, 23, 52, 27, 11]
=== Training estimator # 91 with 7 features
===> Done training estimator # 91 in 15.988931655883789 seconds
--------------------------------------------------------------

Selected features [36, 3, 43, 6, 30, 29, 24]
=== Training estimator # 92 with 7 features
===> Done training estimator # 92 in 9.185672998428345 seconds
--------------------------------------------------------------

Selected features [36, 42, 31, 52, 14, 39, 15]
=== Training estimator # 93 with 7 features
===> Done training estimator # 93 in 10.487852096557617 seconds
--------------------------------------------------------------

Selected features [49, 8, 2, 12, 15, 34, 29]
=== Training estimator # 94 with 7 features
===> Done training estimator # 94 in 15.97676134109497 seconds
--------------------------------------------------------------

Se

In [35]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth=10, n_estimators=10, max_features=7)
rfc.fit(X_test, y_test)
classfierData(rfc, X_test, y_test)
print("-----------------------------------------------------------------------")
rfc = RandomForestClassifier(max_depth=10, n_estimators=50, max_features=7)
rfc.fit(X_test, y_test)
classfierData(rfc, X_test, y_test)
print("-----------------------------------------------------------------------")
rfc = RandomForestClassifier(max_depth=10, n_estimators=100, max_features=7)
rfc.fit(X_test, y_test)
classfierData(rfc, X_test, y_test)

Confusion matrix
 [[694  24]
 [  3 430]] 

True negative: 694 , false positive: 24 , false negative: 3 ,true positive: 430 

Accuracy score 0.9765421372719374 

Precision 0.947136563876652 

Recall 0.9930715935334873 

F1 score 0.9695603156708005 

-----------------------------------------------------------------------
Confusion matrix
 [[697  20]
 [  0 434]] 

True negative: 697 , false positive: 20 , false negative: 0 ,true positive: 434 

Accuracy score 0.9826238053866203 

Precision 0.9559471365638766 

Recall 1.0 

F1 score 0.9774774774774774 

-----------------------------------------------------------------------
Confusion matrix
 [[696  22]
 [  1 432]] 

True negative: 696 , false positive: 22 , false negative: 1 ,true positive: 432 

Accuracy score 0.9800173761946134 

Precision 0.9515418502202643 

Recall 0.9976905311778291 

F1 score 0.9740698985343856 



# Question 2

In [25]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [26]:
ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=50)

In [27]:
ada.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=50, random_state=None)

In [28]:
classfierData(ada, X_test, y_test)

Confusion matrix
 [[668  33]
 [ 29 421]] 

True negative: 668 , false positive: 33 , false negative: 29 ,true positive: 421 

Accuracy score 0.946133796698523 

Precision 0.9273127753303965 

Recall 0.9355555555555556 

F1 score 0.9314159292035398 



In [29]:
from sklearn.linear_model import LogisticRegression
ada = AdaBoostClassifier(LogisticRegression(), n_estimators=50)

In [30]:
ada.fit(X_train, y_train)
classfierData(ada, X_test, y_test)

Confusion matrix
 [[653  69]
 [ 44 385]] 

True negative: 653 , false positive: 69 , false negative: 44 ,true positive: 385 

Accuracy score 0.9018245004344049 

Precision 0.8480176211453745 

Recall 0.8974358974358975 

F1 score 0.8720271800679501 



In [31]:
base_learners = [10, 50, 100]
# use with decision tree
for l in base_learners:
    ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=l)
    ada.fit(X_train, y_train)
    print("AdaBoost classifier data for", l, "number of estimators\n")
    print("*********************************************************************")
    classfierData(ada, X_test, y_test)
    print("---------------------------------------------------------------------")

AdaBoost classifier data for 10 number of estimators

*********************************************************************
Confusion matrix
 [[669  35]
 [ 28 419]] 

True negative: 669 , false positive: 35 , false negative: 28 ,true positive: 419 

Accuracy score 0.945264986967854 

Precision 0.9229074889867841 

Recall 0.9373601789709173 

F1 score 0.93007769145394 

---------------------------------------------------------------------
AdaBoost classifier data for 50 number of estimators

*********************************************************************
Confusion matrix
 [[665  35]
 [ 32 419]] 

True negative: 665 , false positive: 35 , false negative: 32 ,true positive: 419 

Accuracy score 0.9417897480451781 

Precision 0.9229074889867841 

Recall 0.9290465631929047 

F1 score 0.9259668508287293 

---------------------------------------------------------------------
AdaBoost classifier data for 100 number of estimators

**********************************************************

In [36]:
from sklearn.ensemble import RandomForestClassifier

for l in base_learners:
    ada = AdaBoostClassifier(RandomForestClassifier(n_estimators=10), n_estimators=l)
    ada.fit(X_train, y_train)
    print("AdaBoost classifier data for", l, "number of Random Forest estimators\n")
    print("*********************************************************************")
    classfierData(ada, X_test, y_test)
    print("---------------------------------------------------------------------")

AdaBoost classifier data for 10 number of Random Forest estimators

*********************************************************************
Confusion matrix
 [[678  31]
 [ 19 423]] 

True negative: 678 , false positive: 31 , false negative: 19 ,true positive: 423 

Accuracy score 0.9565595134665508 

Precision 0.9317180616740088 

Recall 0.9570135746606335 

F1 score 0.9441964285714286 

---------------------------------------------------------------------
AdaBoost classifier data for 50 number of Random Forest estimators

*********************************************************************
Confusion matrix
 [[676  32]
 [ 21 422]] 

True negative: 676 , false positive: 32 , false negative: 21 ,true positive: 422 

Accuracy score 0.9539530842745438 

Precision 0.9295154185022027 

Recall 0.9525959367945824 

F1 score 0.9409141583054628 

---------------------------------------------------------------------
AdaBoost classifier data for 100 number of Random Forest estimators

*************

# Problem 3

In [38]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.datasets import mnist
from keras.utils import np_utils
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

Using TensorFlow backend.


In [39]:
# Preprocess data for neural net

In [40]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

X_train = np.reshape(X_train, (60000, 784))
X_test = np.reshape(X_test, (10000, 784))

11493376/11490434 [==============================] - 1s 0us/step


## Configuration for FFN

This model will have 3 hidden layer. Each layer (except for the last one to produce result) will use relu as activation functions. After calculating neurons for each layer, Dropout Regularization will be applied

With Keras, it is straightforward and easy to interprete to declare a feed forward neural network (dense network) with `Dense` constructor. With the first model, the hidden layer is a 500 neurons layer and the input layer is the same size as one picture: $28 \times 28$. After that a relu activation function is being applied and a dropout regularization with 0.4 ratio is applied. Next layer is 300 neurons layer and similar relu/dropout being applied. Last layer is the classification layer with 10 neurons as 10 classes being represented and with a softmax activation function. All the three models (networks) use RMS optimizers and based on accuracy. All the metrics are reported on each run. 

In [41]:
model = Sequential()
model.add(Dense(500, input_dim=28*28))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(300))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(10))
model.add(Activation('softmax'))

from keras.optimizers import RMSprop
rms = RMSprop()

In [42]:
def run_network(model, epochs=20, batch=256):
    start_time = time.time()
    model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])
    print('Training model...')
    model.fit(X_train, y_train, nb_epoch=epochs, batch_size=batch,
              validation_data=(X_test, y_test), verbose=2)

    print("Training duration : {0}".format(time.time() - start_time))
    score = model.evaluate(X_test, y_test, batch_size=16)
    print("Network's test score [loss, accuracy]: {0}".format(score))

In [43]:
run_network(model)

Training model...


/home/hoang/Documents/school/4400/env/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 60000 samples, validate on 10000 samples
Epoch 1/20
 - 3s - loss: 0.3588 - acc: 0.8897 - val_loss: 0.1488 - val_acc: 0.9526
Epoch 2/20
 - 2s - loss: 0.1529 - acc: 0.9543 - val_loss: 0.1076 - val_acc: 0.9668
Epoch 3/20
 - 2s - loss: 0.1148 - acc: 0.9650 - val_loss: 0.0896 - val_acc: 0.9725
Epoch 4/20
 - 2s - loss: 0.0948 - acc: 0.9720 - val_loss: 0.0760 - val_acc: 0.9762
Epoch 5/20
 - 2s - loss: 0.0829 - acc: 0.9749 - val_loss: 0.0732 - val_acc: 0.9777
Epoch 6/20
 - 2s - loss: 0.0696 - acc: 0.9785 - val_loss: 0.0699 - val_acc: 0.9803
Epoch 7/20
 - 2s - loss: 0.0627 - acc: 0.9805 - val_loss: 0.0821 - val_acc: 0.9780
Epoch 8/20
 - 2s - loss: 0.0594 - acc: 0.9818 - val_loss: 0.0689 - val_acc: 0.9823
Epoch 9/20
 - 2s - loss: 0.0530 - acc: 0.9837 - val_loss: 0.0706 - val_acc: 0.9807
Epoch 10/20
 - 2s - loss: 0.0502 - acc: 0.9850 - val_loss: 0.0711 - val_acc: 0.9828
Epoch 11/20
 - 2s - loss: 0.0469 - acc: 0.9852 - val_loss: 0.0732 - val_acc: 0.9823
Epoch 12/20
 - 2s - loss: 0.0436 - 

This network is a lot simpler with only 1 output layer of 10 neurons and with a softmax activation

In [44]:
model = Sequential()
model.add(Dense(10, input_dim=28*28))
model.add(Activation('softmax'))

In [45]:
run_network(model)

Training model...
Train on 60000 samples, validate on 10000 samples
Epoch 1/20


/home/hoang/Documents/school/4400/env/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


 - 0s - loss: 0.7553 - acc: 0.8183 - val_loss: 0.3999 - val_acc: 0.8960
Epoch 2/20
 - 0s - loss: 0.3688 - acc: 0.8989 - val_loss: 0.3221 - val_acc: 0.9115
Epoch 3/20
 - 0s - loss: 0.3225 - acc: 0.9103 - val_loss: 0.3008 - val_acc: 0.9162
Epoch 4/20
 - 0s - loss: 0.3028 - acc: 0.9156 - val_loss: 0.2905 - val_acc: 0.9183
Epoch 5/20
 - 0s - loss: 0.2915 - acc: 0.9190 - val_loss: 0.2817 - val_acc: 0.9218
Epoch 6/20
 - 0s - loss: 0.2840 - acc: 0.9209 - val_loss: 0.2773 - val_acc: 0.9231
Epoch 7/20
 - 0s - loss: 0.2782 - acc: 0.9220 - val_loss: 0.2731 - val_acc: 0.9233
Epoch 8/20
 - 0s - loss: 0.2738 - acc: 0.9232 - val_loss: 0.2695 - val_acc: 0.9241
Epoch 9/20
 - 0s - loss: 0.2706 - acc: 0.9243 - val_loss: 0.2697 - val_acc: 0.9246
Epoch 10/20
 - 0s - loss: 0.2674 - acc: 0.9254 - val_loss: 0.2688 - val_acc: 0.9253
Epoch 11/20
 - 0s - loss: 0.2650 - acc: 0.9265 - val_loss: 0.2669 - val_acc: 0.9261
Epoch 12/20
 - 0s - loss: 0.2629 - acc: 0.9272 - val_loss: 0.2659 - val_acc: 0.9273
Epoch 13/20


This FFN is a little bit more complicated than the second one with the first layer as a 300 neurons layer

In [46]:
model = Sequential()
model.add(Dense(300, input_dim=28*28))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

In [47]:
run_network(model)

Training model...
Train on 60000 samples, validate on 10000 samples
Epoch 1/20


/home/hoang/Documents/school/4400/env/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


 - 1s - loss: 0.3398 - acc: 0.9065 - val_loss: 0.1884 - val_acc: 0.9442
Epoch 2/20
 - 1s - loss: 0.1531 - acc: 0.9551 - val_loss: 0.1243 - val_acc: 0.9628
Epoch 3/20
 - 1s - loss: 0.1046 - acc: 0.9692 - val_loss: 0.0931 - val_acc: 0.9723
Epoch 4/20
 - 1s - loss: 0.0785 - acc: 0.9769 - val_loss: 0.0852 - val_acc: 0.9739
Epoch 5/20
 - 1s - loss: 0.0614 - acc: 0.9822 - val_loss: 0.0795 - val_acc: 0.9753
Epoch 6/20
 - 1s - loss: 0.0494 - acc: 0.9854 - val_loss: 0.0706 - val_acc: 0.9770
Epoch 7/20
 - 1s - loss: 0.0405 - acc: 0.9886 - val_loss: 0.0689 - val_acc: 0.9784
Epoch 8/20
 - 1s - loss: 0.0338 - acc: 0.9905 - val_loss: 0.0843 - val_acc: 0.9739
Epoch 9/20
 - 1s - loss: 0.0276 - acc: 0.9923 - val_loss: 0.0723 - val_acc: 0.9781
Epoch 10/20
 - 1s - loss: 0.0223 - acc: 0.9941 - val_loss: 0.0669 - val_acc: 0.9817
Epoch 11/20
 - 1s - loss: 0.0189 - acc: 0.9951 - val_loss: 0.0659 - val_acc: 0.9799
Epoch 12/20
 - 1s - loss: 0.0154 - acc: 0.9962 - val_loss: 0.0677 - val_acc: 0.9796
Epoch 13/20


## Convolutional NN

For Convolutional Neural Network, we will not flatten the $28 \times 28$ image into a $781 \times 1$ vector.

In [48]:
import keras

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

Similar to normal FFN, Conv net in keras is straightforward to declare and also easy to interprete from the code. For example, this NN has the first layer as a 64 neurons with a $2 \times 2$ filter. The input is the same size as the image $28 \times 28 \times 1$ and with relu activation. The next maxpooling layer is having a $2\times 2$ pool size (pooling windows). Similarly, the next Conv Net layer is being declared. Lastly, the output layer has 10 neurons and with a softmax activation to classify the labels.

The next two layers are having the same description so I would not explained further and Keras does a great job at having a declarative API.

In [49]:
convMod = Sequential()

convMod.add(Conv2D(64, (2, 2), input_shape = (28, 28, 1), activation = 'relu'))
convMod.add(MaxPooling2D(pool_size = (2, 2)))

convMod.add(Conv2D(64, (2, 2), activation = 'relu'))
convMod.add(MaxPooling2D(pool_size = (1, 1)))
convMod.add(Dropout(0.2))

convMod.add(Flatten())

convMod.add(Dense(units = 128, activation='relu'))
convMod.add(Dropout(0.5))
convMod.add(Dense(units = 10, activation='softmax'))

In [50]:
convMod.compile(optimizer=rms, loss = 'categorical_crossentropy', metrics = ['accuracy'])
convMod.fit(x_train, y_train,epochs=10, verbose = 1)

Epoch 1/10
60000/60000 [==============================] - 51s 848us/step - loss: 0.1996 - acc: 0.9399
Epoch 2/10
60000/60000 [==============================] - 50s 836us/step - loss: 0.0916 - acc: 0.9737
Epoch 3/10
60000/60000 [==============================] - 50s 839us/step - loss: 0.0795 - acc: 0.9768
Epoch 4/10
60000/60000 [==============================] - 50s 833us/step - loss: 0.0784 - acc: 0.9779
Epoch 5/10
60000/60000 [==============================] - 50s 833us/step - loss: 0.0836 - acc: 0.9768
Epoch 6/10
60000/60000 [==============================] - 50s 837us/step - loss: 0.0816 - acc: 0.9776
Epoch 7/10
60000/60000 [==============================] - 51s 847us/step - loss: 0.0828 - acc: 0.9768
Epoch 8/10
60000/60000 [==============================] - 57s 943us/step - loss: 0.0868 - acc: 0.9768
Epoch 9/10
60000/60000 [==============================] - 57s 944us/step - loss: 0.0908 - acc: 0.9764
Epoch 10/10
60000/60000 [==============================] - 56s 927us/step - loss: 

In [51]:
print(convMod.evaluate(x_test, y_test, batch_size=16))

10000/10000 [==============================] - 2s 213us/step
[0.055363797660581075, 0.9844]


In [57]:
convMod = Sequential()

convMod.add(Conv2D(64, (4, 4), input_shape = (28, 28, 1), activation = 'relu'))
convMod.add(MaxPooling2D(pool_size = (2, 2)))
convMod.add(Dropout(0.2))
convMod.add(Flatten())
convMod.add(Dense(units = 128, activation='relu'))
convMod.add(Dropout(0.5))
convMod.add(Dense(units = 10, activation='softmax'))

In [59]:
convMod.compile(optimizer=rms, loss = 'categorical_crossentropy', metrics = ['accuracy'])
convMod.fit(x_train, y_train,epochs=10, verbose = 1)
print(convMod.evaluate(x_test, y_test, batch_size=16))

Epoch 1/10
60000/60000 [==============================] - 41s 684us/step - loss: 0.0892 - acc: 0.9771
Epoch 2/10
60000/60000 [==============================] - 40s 664us/step - loss: 0.0934 - acc: 0.9761
Epoch 3/10
60000/60000 [==============================] - 42s 707us/step - loss: 0.0959 - acc: 0.9754
Epoch 4/10
60000/60000 [==============================] - 38s 631us/step - loss: 0.0961 - acc: 0.9763
Epoch 5/10
60000/60000 [==============================] - 38s 625us/step - loss: 0.0994 - acc: 0.9755
Epoch 6/10
60000/60000 [==============================] - 41s 679us/step - loss: 0.1000 - acc: 0.9749
Epoch 7/10
60000/60000 [==============================] - 39s 658us/step - loss: 0.1032 - acc: 0.9747
Epoch 8/10
60000/60000 [==============================] - 38s 625us/step - loss: 0.1093 - acc: 0.9738
Epoch 9/10
60000/60000 [==============================] - 38s 633us/step - loss: 0.1097 - acc: 0.9731
Epoch 10/10
10000/10000 [==============================] - 2s 184us/step
[0.084725

In [54]:
convMod = Sequential()

convMod.add(Conv2D(64, (2, 2), input_shape = (28, 28, 1), activation = 'relu'))
convMod.add(MaxPooling2D(pool_size = (1, 1)))
convMod.add(Dropout(0.4))
convMod.add(Flatten())
convMod.add(Dense(units = 32, activation='relu'))
convMod.add(Dropout(0.8))
convMod.add(Dense(units = 10, activation='softmax'))

In [58]:
convMod.compile(optimizer=rms, loss = 'categorical_crossentropy', metrics = ['accuracy'])
convMod.fit(x_train, y_train,epochs=10, verbose = 1)

Epoch 1/10
60000/60000 [==============================] - 39s 657us/step - loss: 0.1990 - acc: 0.9405
Epoch 2/10
60000/60000 [==============================] - 39s 653us/step - loss: 0.0928 - acc: 0.9736
Epoch 3/10
60000/60000 [==============================] - 39s 651us/step - loss: 0.0814 - acc: 0.9771
Epoch 4/10
60000/60000 [==============================] - 39s 649us/step - loss: 0.0791 - acc: 0.9776
Epoch 5/10
60000/60000 [==============================] - 39s 650us/step - loss: 0.0773 - acc: 0.9779
Epoch 6/10
60000/60000 [==============================] - 39s 651us/step - loss: 0.0773 - acc: 0.9782
Epoch 7/10
60000/60000 [==============================] - 39s 650us/step - loss: 0.0783 - acc: 0.9787
Epoch 8/10
60000/60000 [==============================] - 39s 650us/step - loss: 0.0816 - acc: 0.9781
Epoch 9/10
60000/60000 [==============================] - 39s 650us/step - loss: 0.0820 - acc: 0.9782
Epoch 10/10
10000/10000 [==============================] - 2s 199us/step
[0.093589

In [56]:
print(convMod.evaluate(x_test, y_test, batch_size=16))

10000/10000 [==============================] - 2s 190us/step
[0.5539265819072723, 0.9153]


For this specific problem, I think normal feed forward neural network did fine with faster training time and overall accuracy is 98% with just 10 epochs. Conv Net trained longer with this simple images dataset and might be an overkill. I think for a complex dataset, Conv Net will be more useful as it learns the image by applying different filters.